In [2]:
import pandas as pd
import numpy as np

# show all df columns
pd.set_option('display.max_columns', None)

# Helps to reload .py functions when edited
%load_ext autoreload
%autoreload 2

In [3]:
from sklearn.metrics import r2_score

# Define scoring metric
def rmse(np.exp(y_actual), np.exp(y_predicted)):
    return np.sqrt(np.mean(np.square(y_actual - y_predicted)))

def print_score(m):
    y_bar = m.predict(X)
    res = [rmse(y_bar, y), r2_score(y_bar.reshape(-1,1), y.values.reshape(-1,1))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [6]:
train_df = pd.read_pickle("LE_data/train_df.pkl")
test_df = pd.read_pickle("LE_data/test_df.pkl")
target_df = pd.read_pickle("LE_data/target_df.pkl")

In [7]:
train_df.shape, test_df.shape

((175000, 538), (83210, 538))

In [5]:
test_df.head()

,designation,province,region_1,region_2,taster_name,title,variety,winery,ratings,country_region,has_twit_handle,title_year,no_of_words,designation_province,designation_region_1,designation_region_2,designation_taster_name,designation_title,designation_variety,designation_winery,designation_ratings,designation_country_region,designation_has_twit_handle,designation_title_year,designation_no_of_words,province_region_1,province_region_2,province_taster_name,province_title,province_variety,province_winery,province_ratings,province_country_region,province_has_twit_handle,province_title_year,province_no_of_words,region_1_region_2,region_1_taster_name,region_1_title,region_1_variety,region_1_winery,region_1_ratings,region_1_country_region,region_1_has_twit_handle,region_1_title_year,region_1_no_of_words,region_2_taster_name,region_2_title,region_2_variety,region_2_winery,region_2_ratings,region_2_country_region,region_2_has_twit_handle,region_2_title_year,region_2_no_of_words,taster_name_title,taster_name_variety,taster_name_winery,taster_name_ratings,taster_name_country_region,taster_name_has_twit_handle,taster_name_title_year,taster_name_no_of_words,title_variety,title_winery,title_ratings,title_country_region,title_has_twit_handle,title_title_year,title_no_of_words,variety_winery,variety_ratings,variety_country_region,variety_has_twit_handle,variety_title_year,variety_no_of_words,winery_ratings,winery_country_region,winery_has_twit_handle,winery_title_year,winery_no_of_words,ratings_country_region,ratings_has_twit_handle,ratings_title_year,ratings_no_of_words,country_region_has_twit_handle,country_region_title_year,country_region_no_of_words,has_twit_handle_title_year,has_twit_handle_no_of_words,title_year_no_of_words,acidity,age,also,apple,aromas,balance,balanced,berry,bit,black,blackberry,blend,bodied,bright,cabernet,character,chardonnay,cherries,cherry,chocolate,citrus,clean,cola,complex,creamy,crisp,currant,dark,dried,drink,dry,elegant,fine,finish,firm,flavor,flavors,fresh,fruit,fruits,fruity,full,good,green,herb,herbal,juicy,lemon,licorice,light,like,lime,little,long,made,medium,merlot,mineral,mouth,mouthfeel,nose,notes,oak,offers,one,palate,peach,pear,pepper,pinot,plum,raspberry,red,rich,ripe,sauvignon,shows,slightly,smooth,soft,spice,spicy,structure,style,sweet,syrah,tannic,tannins,tart,texture,toast,tobacco,touch,vanilla,vineyard,well,white,wine,wood,years
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.279489,3.427496,4.067866,3.845235,3.268292,3.279489,3.517076,3.903846,5,3.381689,0,2007,57,11.240437,13.340522,12.610408,10.718329,10.755051,11.534213,12.802621,16.397447,11.090213,0.000000,6581.935410,186.930901,13.942592,13.179528,11.202058,11.240437,12.054763,13.380416,17.137480,11.590725,0.000000,6878.984327,195.367268,15.641899,13.294973,13.340522,14.306991,15.880320,20.339328,13.756255,0.000000,8164.206094,231.868335,12.567351,12.610408,13.523983,15.011205,19.226176,13.003389,0.000000,7717.386884,219.178402,10.718329,11.494831,12.758908,16.341460,11.052347,0.000000,6559.462152,186.292647,11.534213,12.802621,16.397447,11.090213,0.000000,6581.935410,186.930901,13.730121,17.585379,11.893656,0.000000,7058.771080,200.473319,19.519229,13.201592,0.000000,7835.018511,222.519210,16.908444,0,10035,285,0.000000,6787.049477,192.756263,0,0,114399,0.00000,0.0,0.000000,0.000000,2.312604,0.0,0.000000,0.000000,3.910662,0.0,3.318894,0.0,0.000000,0.0,3.604391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.066651,0.0,0.0,0.000000,2.90862,0.0,0.0,0.000000,0.0,0.0,3.400044,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,5.981324,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,3.042779,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,3.108811,0.00000,3.945697,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X = train_df.copy()
y = target_df.copy()

scaler.fit(X)
X = scaler.transform(X)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.models import * 
from tensorflow.keras.layers import * 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam

def buildmodel():
    model = Sequential()
    model.add(Dense(100, input_shape=(538, ), activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(Adam(lr=0.05), 'mse', metrics=['accuracy'])
    # Compile model 
    return(model)

In [ ]:
estimator= KerasRegressor(build_fn=buildmodel, epochs=100, batch_size=10, verbose=0)
kfold= RepeatedKFold(n_splits=5, n_repeats=100)
results= cross_val_score(estimator, X, y, cv=kfold, n_jobs=-1)  # 2 cpus
results.mean()  # Mean MSE

In [17]:
test_label = rf1.predict(test_df)

In [18]:
test_label[0:10]

array([4.38321501, 3.68847121, 4.05933296, 3.40577394, 2.42914121,
       2.86780819, 2.42973405, 2.41138805, 2.57679508, 3.93239843])

In [22]:
# Submissions to kaggle
from submission import submit
submit(test_label, test_df)

Fist 10 predictions: [80.09512646058883, 39.98367341546523, 57.935652525062466, 30.13761134153912, 11.349131390153303, 17.598403591851913, 11.355861547103217, 11.149426332275057, 13.15491009869718, 51.029221132667004]

Save path : submissions/submission_rf1.csv
